In [1]:
from data.helper import *

In [2]:
DF = get_plan_df()

100%|██████████| 1200/1200 [00:00<00:00, 18793.02it/s]


In [3]:
for metric in ["tried", "solved"]:
    print(metric)
    df = DF.copy()
    df = df[df.iterations == "4"]
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(["pruning", "optimiser", "domain"]).sum().reset_index()
    df = df[["pruning", "optimiser", "domain", metric]]
    # pivot table
    df = df.pivot_table(index=["pruning", "optimiser"], columns="domain", values=metric)
    # make entries int2
    df = df.astype(int)
    df = df.fillna(0)
    display(df)

tried


domain                    blocksworld  childsnack  ferry  floortile  miconic  \
pruning        optimiser                                                       
collapse-layer rank-svm            30          30     30         30       30   
               svr                 30          30     30         30       30   
none           rank-svm            30          30     30          0       30   
               svr                 30          30     30         30       30   

domain                    rovers  satellite  sokoban  spanner  transport  
pruning        optimiser                                                  
collapse-layer rank-svm        0          0       30       30         30  
               svr            30         30       30       30         30  
none           rank-svm        0          0       30       30         30  
               svr            30         30       30       30         30

solved


domain                    blocksworld  childsnack  ferry  floortile  miconic  \
pruning        optimiser                                                       
collapse-layer rank-svm            23          10     24          1       30   
               svr                 10           7     23          1       30   
none           rank-svm            20          11     23          0       30   
               svr                 19          11     22          1       30   

domain                    rovers  satellite  sokoban  spanner  transport  
pruning        optimiser                                                  
collapse-layer rank-svm        0          0       12       22         11  
               svr            17         18       12       22         10  
none           rank-svm        0          0       12       21         10  
               svr            12         17       12       22          9

In [4]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = DF.copy()
        df = df[df.iterations == "4"]
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        df["config"] = df["pruning"] + " - " + df["optimiser"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)

In [5]:
for domain in DOMAINS:
    df = DF.copy()
    print(domain)
    df = df[df['domain'] == domain]
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(['pruning', 'optimiser', 'iterations']).sum().reset_index()
    df = df[['pruning', 'optimiser', 'iterations', 'solved']]
    # pivot table
    df = df.pivot_table(index=['pruning', 'optimiser'], columns='iterations', values='solved')
    # make entries int2
    df = df.astype(int)
    df = df.fillna(0)
    display(df)

blocksworld


iterations                 4
pruning        optimiser    
collapse-layer rank-svm   23
               svr        10
none           rank-svm   20
               svr        19

childsnack


iterations                 4
pruning        optimiser    
collapse-layer rank-svm   10
               svr         7
none           rank-svm   11
               svr        11

ferry


...

iterations                 4
pruning        optimiser    
collapse-layer rank-svm   12
               svr        12
none           rank-svm   12
               svr        12

spanner


iterations                 4
pruning        optimiser    
collapse-layer rank-svm   22
               svr        22
none           rank-svm   21
               svr        22

transport


iterations                 4
pruning        optimiser    
collapse-layer rank-svm   11
               svr        10
none           rank-svm   10
               svr         9